<a href="https://colab.research.google.com/github/henriquevedoveli/correcao-provas/blob/master/word2vec_treinamento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classificando Títulos de Notícias Utilizando Word2Vec
### ***Notebook de Treinamento do Modelo***

---

## ***Objetivo***
#### Desenvolver um classificador capaz de realizar automaticamente o processo de categorização de títulos de notícias.

## Download do core em português (rodar apenas uma vez)

In [1]:
# !python -m spacy download pt_core_news_sm

Bibliotecas Necessárias

In [2]:
import pandas as pd
import spacy
from gensim.models import Word2Vec
import logging

## Importando Dados do Drive

In [3]:
dados_treino = pd.read_csv('/content/drive/My Drive/data_science/classificacao-titulos/dados/treino.csv')

dados_treino.sample(5)

,title,text,date,category,subcategory,link
66266,Brasil con Ñ: Medidas de película contra una s...,POR LUNA GÁMEZ Y JOSÉ BAUTISTA A Brasil se le...,2015-11-02,mundo,NaN,http://www1.folha.uol.com.br/mundo/2015/02/158...
17164,Doping esquenta os bastidores no final da temp...,Por apontar falhas no controle antidoping do m...,2016-01-11,colunas,edgardalves,http://www1.folha.uol.com.br/colunas/edgardalv...
32147,Presidente do Palmeiras desafia clubes da Chin...,Em meio à transferências de jogadores do Corin...,2016-01-18,esporte,NaN,http://www1.folha.uol.com.br/esporte/2016/01/1...
28143,Temer terá curta lua de mel com mercados até e...,Com a aprovação do pedido de impeachment na Câ...,2016-04-18,mercado,NaN,http://www1.folha.uol.com.br/mercado/2016/04/1...
81446,"'Chega de Merkel', diz opositor uma semana ant...",A quatro semanas da eleição alemã e uma semana...,2017-08-28,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/08/191...


## Construindo o Vocabulário

In [4]:
nlp = spacy.load('pt_core_news_sm')

In [5]:
texto = "Um texto qualquer para o teste"
doc = nlp(texto)

---
## Pré-Processamento dos Dados
### Definindo uma função para tratar os textos
#### Todas as letras maiúsculas do textos do Dataset serão transformadas em letras minúsculas.

#### Casos os textos não sejam uma stop word e nem alfanúmerico, o token será considerado um token válido, caso seja uma stop word ou um alfanúmerico o token será excluido.

In [6]:
textos_para_tratamento = (titulos.lower() for titulos in dados_treino['title'])

In [7]:
def trata_textos(doc):
  tokens_validos = []
  for token in doc:
    eh_valido = not token.is_stop and token.is_alpha
    if eh_valido:
      tokens_validos.append(token.text)

  if len(tokens_validos) > 2:
    return ' '.join(tokens_validos)

In [8]:
trata_textos(doc)

'texto o teste'

In [9]:
textos_tratados = [trata_textos(doc) for doc in nlp.pipe(textos_para_tratamento,
                                                        batch_size = 1000,
                                                        n_process = -1)]

In [10]:
titulos_tratados = pd.DataFrame({'titulo':textos_tratados})
titulos_tratados = titulos_tratados.dropna().drop_duplicates()

titulos_tratados.sample(5)

,titulo
68958,árvore porte cai calçada e obriga pedestre a t...
40068,frases livro reúne a ironia e o pragmatismo de...
74456,principal força política argentina peronismo c...
70508,botafogo renova jefferson e vaquinha pagar dívida
9410,garantir erundina debates tv psol discute alia...


## Definindo o Modelo Word2Vec

#### Em um modelo Word2Vec convertemos palavras em vetores em podemos medir a distância entre eles sendo assim, palavras semelhantes são mais próximas que palavras diferentes. 

#### O Word2Vec se tornou um dos métodos mais utilizados em pré-processamento de dados em formato de textos para realizar tarefas de Processamento de Linguagem Natural. O método foi desenvolvido por Tomas Mikolov em 2013 como uma respostas para tornar o treinamento de embeddings baseadas em redes neurais mais eficaz.

#### O artigo original publicado por Tomas Mikolov pode ser acessado pelo [link](https://arxiv.org/pdf/1301.3781.pdf).

In [11]:
lista_lista_token = [titulo.split(' ') for titulo in titulos_tratados['titulo']]

### Construindo Modelo CBOW

In [12]:
w2v_model_cbow = Word2Vec(sg = 0,
                     window = 2,
                     size = 300,
                     min_count = 5,
                     alpha = 0.03,
                     min_alpha = 0.007)

logging.basicConfig(format='%(asctime)s : - %(message)s', level = logging.INFO)
w2v_model_cbow.build_vocab(lista_lista_token, progress_per=5000)

2020-08-12 17:46:06,105 : - collecting all words and their counts
2020-08-12 17:46:06,108 : - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-08-12 17:46:06,125 : - PROGRESS: at sentence #5000, processed 34716 words, keeping 10129 word types
2020-08-12 17:46:06,136 : - PROGRESS: at sentence #10000, processed 69298 words, keeping 14909 word types
2020-08-12 17:46:06,150 : - PROGRESS: at sentence #15000, processed 103841 words, keeping 18223 word types
2020-08-12 17:46:06,166 : - PROGRESS: at sentence #20000, processed 138620 words, keeping 20969 word types
2020-08-12 17:46:06,178 : - PROGRESS: at sentence #25000, processed 173257 words, keeping 23410 word types
2020-08-12 17:46:06,193 : - PROGRESS: at sentence #30000, processed 207976 words, keeping 25453 word types
2020-08-12 17:46:06,210 : - PROGRESS: at sentence #35000, processed 242567 words, keeping 27263 word types
2020-08-12 17:46:06,223 : - PROGRESS: at sentence #40000, processed 277254 words, keeping 2899

### Construindo Modelo Skip-Gram

In [13]:
w2v_model_sg = Word2Vec(sg = 1,
                     window = 5,
                     size = 300,
                     min_count = 5,
                     alpha = 0.03,
                     min_alpha = 0.007)

logging.basicConfig(format='%(asctime)s : - %(message)s', level = logging.INFO)
w2v_model_sg.build_vocab(lista_lista_token, progress_per=5000)

2020-08-12 17:46:09,203 : - collecting all words and their counts
2020-08-12 17:46:09,205 : - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-08-12 17:46:09,225 : - PROGRESS: at sentence #5000, processed 34716 words, keeping 10129 word types
2020-08-12 17:46:09,235 : - PROGRESS: at sentence #10000, processed 69298 words, keeping 14909 word types
2020-08-12 17:46:09,248 : - PROGRESS: at sentence #15000, processed 103841 words, keeping 18223 word types
2020-08-12 17:46:09,261 : - PROGRESS: at sentence #20000, processed 138620 words, keeping 20969 word types
2020-08-12 17:46:09,277 : - PROGRESS: at sentence #25000, processed 173257 words, keeping 23410 word types
2020-08-12 17:46:09,288 : - PROGRESS: at sentence #30000, processed 207976 words, keeping 25453 word types
2020-08-12 17:46:09,301 : - PROGRESS: at sentence #35000, processed 242567 words, keeping 27263 word types
2020-08-12 17:46:09,317 : - PROGRESS: at sentence #40000, processed 277254 words, keeping 2899

## Arquitetura do Word2Vec
#### O modelo Word2Vec utiliza de dois modelos em sua arquitetura, os quais possuem uma camada de entrada, uma camada oculta e uma camada de saída.

---
## Treinamento Word2Vec utilizando Continuous Bag of Words (CBOW)

#### O modelo de arquitetura CBOW é utilizado para descobrir a palavra central de uma sentença, baseado nas palavras que a cercam. Utilzando como exemplo a frase ```o cachorro correu atrás do gato``` pode ser representada como:

1. ```([o, correu], cachorro)```

2. ```([cachorro, atrás], correu)```

3. ```([correu, do], atrás)```

4. ```([atrás, gato], do)```

#### Com estes dados, conseguimos "ensinar" ao modelo a predizer uma palavra alvo, baseada em palavras de contexto. O modelo CBOW pode ser representado como na imagem abaixo, onde a camada de entrada receberá os dados pré-processados, como acima, e sua saída terá uma palavra que representa a maior probalidade para a palavra alvo.
![](img/cbow.png)


In [14]:
w2v_model_cbow.train(lista_lista_token,
                total_examples = w2v_model_cbow.corpus_count,
                epochs = 30)

2020-08-12 17:46:12,221 : - training model with 3 workers on 13006 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=2
2020-08-12 17:46:13,273 : - EPOCH 1 - PROGRESS: at 65.26% examples, 318031 words/s, in_qsize 5, out_qsize 0
2020-08-12 17:46:13,726 : - worker thread finished; awaiting finish of 2 more threads
2020-08-12 17:46:13,735 : - worker thread finished; awaiting finish of 1 more threads
2020-08-12 17:46:13,754 : - worker thread finished; awaiting finish of 0 more threads
2020-08-12 17:46:13,755 : - EPOCH - 1 : training on 597929 raw words (502972 effective words) took 1.5s, 332445 effective words/s
2020-08-12 17:46:14,832 : - EPOCH 2 - PROGRESS: at 66.94% examples, 317041 words/s, in_qsize 6, out_qsize 2
2020-08-12 17:46:15,232 : - worker thread finished; awaiting finish of 2 more threads
2020-08-12 17:46:15,262 : - worker thread finished; awaiting finish of 1 more threads
2020-08-12 17:46:15,265 : - worker thread finished; awaiting finish of 0 more t

(15085642, 17937870)

--- 
## Treinamento Word2Vec utilizando Skip-Gram

#### Skip-Gram é o segundo modelo utilizado no Word2Vec, porém neste modelo ao invés de de tentarmos descobrir a palavra central, faremos o processo inverso; da palavra central, tentaremos descobrir as palavras de contexto.
#### Utilizando a mesma frase como exemplo ```o cachorro correu atrás do gato```, obtemos:

1. ```(cachorro, [O, correu])```

2. ```(correu, [cachorro, atrás]))```

3. ```(atrás, [correu, do])```

4. ```(do, [atrás, gato])```

#### Ao contrário da arquitetura do modelo CBOW, a arquitetura do modelo Skip-Gram, a camada de entrada temos apenas a palavra alvo, e em sua camada de saída temos a probalidadem cada uma contendo palavras de contexto possíveis.
![](img/skip-gram.png)


In [15]:
w2v_model_sg.train(lista_lista_token,
                total_examples = w2v_model_sg.corpus_count,
                epochs = 30)

2020-08-12 17:46:56,601 : - training model with 3 workers on 13006 vocabulary and 300 features, using sg=1 hs=0 sample=0.001 negative=5 window=5
2020-08-12 17:46:57,622 : - EPOCH 1 - PROGRESS: at 25.13% examples, 126197 words/s, in_qsize 5, out_qsize 0
2020-08-12 17:46:58,692 : - EPOCH 1 - PROGRESS: at 55.26% examples, 133969 words/s, in_qsize 5, out_qsize 0
2020-08-12 17:46:59,724 : - EPOCH 1 - PROGRESS: at 85.30% examples, 138182 words/s, in_qsize 4, out_qsize 1
2020-08-12 17:47:00,102 : - worker thread finished; awaiting finish of 2 more threads
2020-08-12 17:47:00,135 : - worker thread finished; awaiting finish of 1 more threads
2020-08-12 17:47:00,147 : - worker thread finished; awaiting finish of 0 more threads
2020-08-12 17:47:00,149 : - EPOCH - 1 : training on 597929 raw words (502896 effective words) took 3.5s, 142568 effective words/s
2020-08-12 17:47:01,254 : - EPOCH 2 - PROGRESS: at 28.48% examples, 130866 words/s, in_qsize 5, out_qsize 0
2020-08-12 17:47:02,312 : - EPOCH 2

(15086207, 17937870)

---
### Avaliando Qualitativamento o Modelo

#### Podemos ver que quando pesquisamos as palavras **"google"** e **"barcelona"** as palavras mais similiares com **google** são **apple, facebook, amazon e uber**, que são empresas grandes de tecnologias e quando pesquisando **barcelona** são **barça, chelsea, bayern, psg e figuerense**, sendo barça o mesmo que barcelona, e os outros resultados são todos times de futebol. 

In [16]:
w2v_model_cbow.wv.most_similar('google')

2020-08-12 17:48:39,952 : - precomputing L2-norms of word weight vectors
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('apple', 0.5718475580215454),
 ('facebook', 0.5482473373413086),
 ('fbi', 0.4752236008644104),
 ('amazon', 0.4692961573600769),
 ('uber', 0.4642491042613983),
 ('walmart', 0.4600902497768402),
 ('buffett', 0.454828143119812),
 ('software', 0.4541092813014984),
 ('airbnb', 0.4497571289539337),
 ('sabmiller', 0.4454740583896637)]

In [17]:
w2v_model_cbow.wv.most_similar('barcelona')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('barça', 0.6111436486244202),
 ('bayern', 0.5928974151611328),
 ('chelsea', 0.5928400754928589),
 ('psg', 0.5867067575454712),
 ('juventus', 0.5729324817657471),
 ('madrid', 0.561914324760437),
 ('leicester', 0.5534391403198242),
 ('munique', 0.5455915927886963),
 ('figueirense', 0.5431289672851562),
 ('suárez', 0.54259192943573)]

---
## Salvando o Modelo

In [18]:
w2v_model_cbow.wv.save_word2vec_format('/content/drive/My Drive/data_science/classificacao-titulos/model_cbow.txt', binary = False)

2020-08-12 17:48:40,023 : - storing 13006x300 projection weights into /content/drive/My Drive/data_science/classificacao-titulos/model_cbow.txt
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [19]:
w2v_model_sg.wv.save_word2vec_format('/content/drive/My Drive/data_science/classificacao-titulos/model_sg.txt', binary = False)

2020-08-12 17:49:12,892 : - storing 13006x300 projection weights into /content/drive/My Drive/data_science/classificacao-titulos/model_sg.txt
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
